In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs

### Web Scraping #1: Getting a list of best selling GPUs from the newegg.ca

In [2]:
# this is getting the source code
page_list = [i for i in range(1,4)]
brand = []
name = []
price_list = []
shipping_list = []

# looping through 3 pages
for page in page_list:
    newegg_url = requests.get('https://www.newegg.ca/Desktop-Graphics-Cards/SubCategory/ID-48/Page-{}?Tid=7709&Order=3'.format(page))
    # turn it into a beatifulsoup object - lxml is the parser
    soup = bs.BeautifulSoup(newegg_url.text, 'lxml')
    # grab each product 
    containers = soup.findAll('div', {'class':'item-container'})
    price = soup.findAll('li', {'class':'price-current'})
    shipping = soup.findAll('li', {'class':'price-ship'})

    # getting names
    for container in containers:
        if container.div.div.a.img != None:
            brand.append(container.div.div.a.img['title'])
            name.append(container.a.img['title'])

    # getting prices
    for item in price:
        price_list.append(item.strong.text + item.sup.text)

    # getting prices of shipping
    for item in shipping:
        shipping_list.append(item.text.split()[0])

print(len(shipping_list))
print(len(price_list))
print(len(brand))
print(len(name))

108
108
108
108


In [3]:
price_list = [float(x.replace(',','')) for x in price_list]

In [4]:
frame = {'Name':name, 'Brand':brand, 'Price':price_list, 'Shipping':shipping_list}
gpu_df = pd.DataFrame(frame)
print(gpu_df.head(15))

                                                 Name     Brand   Price  \
0   GIGABYTE GeForce RTX 2070 DirectX 12 GV-N2070W...  GIGABYTE  549.99   
1   MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660...       MSI  338.99   
2   MSI GeForce GTX 1660 DirectX 12 GTX 1660 VENTU...       MSI  289.99   
3   EVGA GeForce RTX 2060 KO ULTRA GAMING Video Ca...      EVGA  459.99   
4   MSI GeForce RTX 2070 SUPER DirectX 12 RTX 2070...       MSI  789.99   
5   MSI GeForce RTX 2060 DirectX 12 RTX 2060 GAMIN...       MSI  519.99   
6   MSI GeForce RTX 2070 DirectX 12 RTX 2070 ARMOR...       MSI  559.99   
7   MSI GeForce RTX 2060 DirectX 12 RTX 2060 VENTU...       MSI  459.99   
8   GIGABYTE GeForce GTX 1650 SUPER WINDFORCE OC 4...  GIGABYTE  229.99   
9   GIGABYTE Radeon RX 580 GAMING 8G (rev. 2.0) Gr...  GIGABYTE  224.99   
10  MSI GeForce RTX 2070 DirectX 12 RTX 2070 VENTU...       MSI  609.99   
11  EVGA GeForce RTX 2070 SUPER KO GAMING Video Ca...      EVGA  704.99   
12  GIGABYTE GeForce GTX 

These are the 15 most sold GPUs on Newegg.ca

### Web Scraping #2: Steam Sale

In [5]:
# getting the source code
steam_url = requests.get('https://store.steampowered.com/search/?specials={}')
soup = bs.BeautifulSoup(steam_url.text, 'lxml')
all_container = soup.findAll('div', {'class':'responsive_search_name_combined'})

game_list = []
discount_list = []
original_price = []
dis_price_list = []

for container in all_container:
    game = container.findChildren()[1].text
    game_list.append(game)
    
    # percentage of discount 
    discount = container.findChildren('div')[3]
    if discount.div.span == None:
        discount_list.append('No Discount')
    else:
        discount_list.append(discount.div.span.text.replace('-',''))
    
    # original price
    o_price = container.findChildren('div')[3].findChildren('div')[1]
    if o_price.strike == None:
        original_price.append('0')
    else:
        original_price.append(o_price.strike.text.split()[1])
    
    # discounted price
    dis_price_line = container.findChildren('div')[3].findChildren('div')[1]
    line_split = dis_price_line.text.strip().split()
    if not line_split:
        dis_price_list.append('0')
    else:
        dis_price_list.append(line_split[-1])

# checking if the number of items match
print(len(dis_price_list))
print(len(original_price))
print(len(discount_list))
print(len(game_list))

50
50
50
50


In [6]:
# to calculate the discounted amount, convert original price list and discounted price list to numpy arrays and do subtraction
original_price = np.array([float(x) for x in original_price])
dis_price_list = np.array([float(x) for x in dis_price_list])

frame = {'Game':game_list, 'Original Price':original_price, 'Discounted Amount':original_price-dis_price_list,
         'Discounted Price':dis_price_list, 'Discount %':discount_list}
steam_df = pd.DataFrame(frame)
steam_df.sort_values(by='Discounted Amount', inplace=True, ascending=False)
steam_df.reset_index(drop=True, inplace=True)
steam_df.head(20)

,Game,Original Price,Discounted Amount,Discounted Price,Discount %
0,More Spore,80.97,58.37,22.60,72%
1,F1® 2019,68.99,51.75,17.24,75%
2,DiRT Rally 2.0,45.49,34.12,11.37,75%
3,Bloodstained: Ritual of the Night,53.49,21.40,32.09,40%
4,Barotrauma Supporter Bundle,46.98,21.19,25.79,45%
5,SPORE™,26.99,20.25,6.74,75%
6,Red Dead Redemption 2,79.99,16.00,63.99,20%
7,Barotrauma,38.99,15.60,23.39,40%
8,SPORE™ Creepy & Cute Parts Pack,26.99,13.50,13.49,50%
9,Cooking Simulator Complete Bundle!,44.27,8.37,35.90,19%


These are the 20 games that give the most value from the sale